In [19]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [20]:
CSV_COLUMN_NAMES =['SepalLength', 'Sepalwidth','PetalLength','PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']


In [21]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

test_path = tf.keras.utils.get_file(
    "iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names = CSV_COLUMN_NAMES,header=0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header=0)


In [22]:
train.head()

,SepalLength,Sepalwidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [23]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

,SepalLength,Sepalwidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [24]:
train.shape

(120, 4)

In [25]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

### Input function

In [26]:
def input_fn(features, labels, training=True, batch_size=256):
  #convert the input to a Dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #shuffle and repeat if you are in training mode
  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

In [27]:
# Define feature columns
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Sepalwidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# Building Model

In [28]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[30, 10],
    n_classes=3
)

In [29]:
# Train the model
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000
)



In [31]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))



Test set accuracy: 0.867



# Making Predictions


In [32]:
def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'Sepalwidth', 'PetalLength', 'PetalWidth']
predict={}

print("please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid:
    val = input(feature + ":")
    if not val.isdigit(): valid= False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda:input_fn(predict))
for pred_dict in predictions:
  print(pred_dict)
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Production is "{}" ({:.1f}%)'.format(
      SPECIES[class_id], 100* probability
    ))

please type numeric values as prompted.
SepalLength:6.0
Sepalwidth:2.4
PetalLength:1.7
PetalWidth:2.5


Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


{'logits': array([ 0.44424763,  0.86164105, -1.2641742 ], dtype=float32), 'probabilities': array([0.37048686, 0.5623989 , 0.06711423], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Production is "Versicolor" (56.2%)
